In [1]:
#Install high-performance libraries
# ! pip install lightgbm polars scikit-learn
import pandas as pd
import polars as pl

In [2]:
#Load CSV using Polars (faster than pandas)
df = pl.read_csv('../../Resources/full_loan_data.csv',
    infer_schema_length=10000,  # helps Polars scan more rows to infer types
    schema_overrides={
        "annual_inc": pl.Float64,
        "funded_amnt_inv" : pl.Float64
        # You can add more overrides here if needed
    }
)

#Convert to pandas if needed for sklearn/lightgbm
df = df.to_pandas()

In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,None,None,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,None,None,Cash,N,None,None,None,None,None,None
1,None,None,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,None,None,Cash,N,None,None,None,None,None,None
2,None,None,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,None,None,Cash,N,None,None,None,None,None,None
3,None,None,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,None,None,Cash,N,None,None,None,None,None,None
4,None,None,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,None,None,Cash,N,None,None,None,None,None,None


In [4]:
columns_to_keep = [
    "member_id",
    "acc_now_delinq",
    "all_util",
    "annual_inc",
    "annual_inc_joint",
    "avg_cur_bal",
    "chargeoff_within_12_mths",
    "delinq_2yrs",
    "delinq_amnt",
    "dti",
    "dti_joint",
    "emp_length",
    "funded_amnt",
    "home_ownership",
    "int_rate",
    "mths_since_last_delinq",
    "num_tl_90g_dpd_24m",
    "open_acc",
    "pct_tl_nvr_dlq",
    "percent_bc_gt_75",
    "pub_rec_bankruptcies",
    "purpose",
    "term",
    "tot_hi_cred_lim",
    "total_acc",
    "total_bal_ex_mort",
    "total_bc_limit",
    "verification_status",
    "verification_status_joint",
    "sec_app_chargeoff_within_12_mths",
    "loan_status",
    "grade"
]

df_subset = df[columns_to_keep]
df_subset.head()

,member_id,acc_now_delinq,all_util,annual_inc,annual_inc_joint,avg_cur_bal,chargeoff_within_12_mths,delinq_2yrs,delinq_amnt,dti,...,term,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bc_limit,verification_status,verification_status_joint,sec_app_chargeoff_within_12_mths,loan_status,grade
0,None,0.0,28.0,55000.0,NaN,1878.0,0.0,0.0,0.0,18.24,...,36 months,60124.0,34.0,16901.0,36500.0,Not Verified,None,NaN,Current,C
1,None,0.0,57.0,90000.0,NaN,24763.0,0.0,0.0,0.0,26.52,...,60 months,372872.0,44.0,99468.0,15000.0,Source Verified,None,NaN,Current,D
2,None,0.0,35.0,59280.0,NaN,18383.0,0.0,0.0,0.0,10.51,...,36 months,136927.0,13.0,11749.0,13800.0,Source Verified,None,NaN,Current,D
3,None,0.0,70.0,92000.0,NaN,30505.0,0.0,0.0,0.0,16.74,...,36 months,385183.0,13.0,36151.0,5000.0,Source Verified,None,NaN,Current,D
4,None,0.0,54.0,57250.0,NaN,9667.0,0.0,0.0,0.0,26.35,...,60 months,157548.0,26.0,29674.0,9300.0,Not Verified,None,NaN,Current,C


In [5]:
# Replace NaN values in the 'member_id' column with sequential numbers
df_subset['member_id'] = df_subset['member_id'].fillna(
    value=pd.Series(range(1, df_subset['member_id'].isnull().sum() + 1))
)

# Display the updated DataFrame
df_subset.head()

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\2430523993.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_subset['member_id'] = df_subset['member_id'].fillna(
C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\2430523993.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['member_id'] = df_subset['member_id'].fillna(


,member_id,acc_now_delinq,all_util,annual_inc,annual_inc_joint,avg_cur_bal,chargeoff_within_12_mths,delinq_2yrs,delinq_amnt,dti,...,term,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bc_limit,verification_status,verification_status_joint,sec_app_chargeoff_within_12_mths,loan_status,grade
0,1,0.0,28.0,55000.0,NaN,1878.0,0.0,0.0,0.0,18.24,...,36 months,60124.0,34.0,16901.0,36500.0,Not Verified,None,NaN,Current,C
1,2,0.0,57.0,90000.0,NaN,24763.0,0.0,0.0,0.0,26.52,...,60 months,372872.0,44.0,99468.0,15000.0,Source Verified,None,NaN,Current,D
2,3,0.0,35.0,59280.0,NaN,18383.0,0.0,0.0,0.0,10.51,...,36 months,136927.0,13.0,11749.0,13800.0,Source Verified,None,NaN,Current,D
3,4,0.0,70.0,92000.0,NaN,30505.0,0.0,0.0,0.0,16.74,...,36 months,385183.0,13.0,36151.0,5000.0,Source Verified,None,NaN,Current,D
4,5,0.0,54.0,57250.0,NaN,9667.0,0.0,0.0,0.0,26.35,...,60 months,157548.0,26.0,29674.0,9300.0,Not Verified,None,NaN,Current,C


In [6]:
df_subset.set_index('member_id', inplace=True)
df_subset.head()

,acc_now_delinq,all_util,annual_inc,annual_inc_joint,avg_cur_bal,chargeoff_within_12_mths,delinq_2yrs,delinq_amnt,dti,dti_joint,...,term,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bc_limit,verification_status,verification_status_joint,sec_app_chargeoff_within_12_mths,loan_status,grade
member_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,28.0,55000.0,NaN,1878.0,0.0,0.0,0.0,18.24,NaN,...,36 months,60124.0,34.0,16901.0,36500.0,Not Verified,None,NaN,Current,C
2,0.0,57.0,90000.0,NaN,24763.0,0.0,0.0,0.0,26.52,NaN,...,60 months,372872.0,44.0,99468.0,15000.0,Source Verified,None,NaN,Current,D
3,0.0,35.0,59280.0,NaN,18383.0,0.0,0.0,0.0,10.51,NaN,...,36 months,136927.0,13.0,11749.0,13800.0,Source Verified,None,NaN,Current,D
4,0.0,70.0,92000.0,NaN,30505.0,0.0,0.0,0.0,16.74,NaN,...,36 months,385183.0,13.0,36151.0,5000.0,Source Verified,None,NaN,Current,D
5,0.0,54.0,57250.0,NaN,9667.0,0.0,0.0,0.0,26.35,NaN,...,60 months,157548.0,26.0,29674.0,9300.0,Not Verified,None,NaN,Current,C


In [7]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2260668 entries, 1 to 2260668
Data columns (total 31 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   acc_now_delinq                    float64
 1   all_util                          float64
 2   annual_inc                        float64
 3   annual_inc_joint                  float64
 4   avg_cur_bal                       float64
 5   chargeoff_within_12_mths          float64
 6   delinq_2yrs                       float64
 7   delinq_amnt                       float64
 8   dti                               float64
 9   dti_joint                         float64
 10  emp_length                        object 
 11  funded_amnt                       int64  
 12  home_ownership                    object 
 13  int_rate                          float64
 14  mths_since_last_delinq            float64
 15  num_tl_90g_dpd_24m                float64
 16  open_acc                          float64

In [8]:
# Merge the joint columns based on the condition
df_subset['merged_annual_inc'] = df_subset['annual_inc_joint'].combine_first(df_subset['annual_inc'])
df_subset['merged_verification_status'] = df_subset['verification_status_joint'].combine_first(df_subset['verification_status'])
df_subset['merged_dti'] = df_subset['dti_joint'].combine_first(df_subset['dti'])


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\852560273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['merged_annual_inc'] = df_subset['annual_inc_joint'].combine_first(df_subset['annual_inc'])
C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\852560273.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['merged_verification_status'] = df_subset['verification_status_joint'].combine_first(df_subset['verification_status'])
C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\8525602

In [9]:
df_subset.drop(columns= ['annual_inc_joint', 'verification_status_joint', 'dti_joint'], inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\3003605293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns= ['annual_inc_joint', 'verification_status_joint', 'dti_joint'], inplace=True)


In [10]:
df_subset.head()

,acc_now_delinq,all_util,annual_inc,avg_cur_bal,chargeoff_within_12_mths,delinq_2yrs,delinq_amnt,dti,emp_length,funded_amnt,...,total_acc,total_bal_ex_mort,total_bc_limit,verification_status,sec_app_chargeoff_within_12_mths,loan_status,grade,merged_annual_inc,merged_verification_status,merged_dti
member_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,28.0,55000.0,1878.0,0.0,0.0,0.0,18.24,10+ years,2500,...,34.0,16901.0,36500.0,Not Verified,NaN,Current,C,55000.0,Not Verified,18.24
2,0.0,57.0,90000.0,24763.0,0.0,0.0,0.0,26.52,10+ years,30000,...,44.0,99468.0,15000.0,Source Verified,NaN,Current,D,90000.0,Source Verified,26.52
3,0.0,35.0,59280.0,18383.0,0.0,0.0,0.0,10.51,6 years,5000,...,13.0,11749.0,13800.0,Source Verified,NaN,Current,D,59280.0,Source Verified,10.51
4,0.0,70.0,92000.0,30505.0,0.0,0.0,0.0,16.74,10+ years,4000,...,13.0,36151.0,5000.0,Source Verified,NaN,Current,D,92000.0,Source Verified,16.74
5,0.0,54.0,57250.0,9667.0,0.0,0.0,0.0,26.35,10+ years,30000,...,26.0,29674.0,9300.0,Not Verified,NaN,Current,C,57250.0,Not Verified,26.35


In [11]:
df_subset.purpose.value_counts()

purpose
debt_consolidation    1277877
credit_card            516971
home_improvement       150457
other                  139440
major_purchase          50445
medical                 27488
small_business          24689
car                     24013
vacation                15525
moving                  15403
house                   14136
wedding                  2355
renewable_energy         1445
educational               424
Name: count, dtype: int64

In [12]:
# Define the mapping of merged categories
purpose_mapping = {
    'debt_consolidation': 'Debt-related',
    'credit_card': 'Debt-related',
    'home_improvement': 'Home-related',
    'house': 'Home-related',
    'moving': 'Home-related',
    'major_purchase': 'Purchase-related',
    'car': 'Purchase-related',
    'wedding': 'Purchase-related',
    'small_business': 'Business/Professional',
    'educational': 'Business/Professional',
    'vacation': 'Lifestyle-related',
    'medical': 'Lifestyle-related',
    'renewable_energy': 'Lifestyle-related',
    'other': 'Other'
}

# Apply the mapping to the 'purpose' column
df_subset['merged_purpose'] = df_subset['purpose'].map(purpose_mapping)

# View the updated DataFrame
df_subset.merged_purpose.value_counts()

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\506060701.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['merged_purpose'] = df_subset['purpose'].map(purpose_mapping)


merged_purpose
Debt-related             1794848
Home-related              179996
Other                     139440
Purchase-related           76813
Lifestyle-related          44458
Business/Professional      25113
Name: count, dtype: int64

In [13]:
df_subset.emp_length.value_counts()

emp_length
10+ years    748005
2 years      203677
< 1 year     189988
3 years      180753
1 year       148403
n/a          146907
5 years      139698
4 years      136605
6 years      102628
7 years       92695
8 years       91914
9 years       79395
Name: count, dtype: int64

In [14]:
def group_emp_length(length):
    if pd.isnull(length) or length == "n/a":  # Handle missing values and 'n/a'
        return "Unknown"
    elif "< 1" in length or "1 year" in length or "2 years" in length or "3 years" in length:
        return "0-3 years"
    elif "4 years" in length or "5 years" in length or "6 years" in length:
        return "4-6 years"
    elif "7 years" in length or "8 years" in length or "9+" in length:
        return "7-9 years"
    elif "10+ years": return "10+ years"

# Apply grouping to the column
df_subset['emp_length_grouped'] = df_subset['emp_length'].apply(group_emp_length)

# View the updated DataFrame
print(df_subset[['emp_length', 'emp_length_grouped']].head())

df_subset.emp_length_grouped.value_counts()

          emp_length emp_length_grouped
member_id                              
1          10+ years          10+ years
2          10+ years          10+ years
3            6 years          4-6 years
4          10+ years          10+ years
5          10+ years          10+ years


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1482086170.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['emp_length_grouped'] = df_subset['emp_length'].apply(group_emp_length)


emp_length_grouped
10+ years    827400
0-3 years    722821
4-6 years    378931
7-9 years    184609
Unknown      146907
Name: count, dtype: int64

In [15]:
df_subset['merged_verification_status'].value_counts()

merged_verification_status
Source Verified    888250
Not Verified       759170
Verified           613248
Name: count, dtype: int64

In [16]:
verification_mapping = {
    'Source Verified': 'Verified',
    'Verified': 'Verified',
    'Not Verified' : 'Not Verified'
}


# Apply the mapping to the 'verification' column
df_subset['clean_verification_status'] = df_subset['merged_verification_status'].map(verification_mapping)

# View the updated DataFrame
df_subset.clean_verification_status.value_counts()

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\2133462108.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['clean_verification_status'] = df_subset['merged_verification_status'].map(verification_mapping)


clean_verification_status
Verified        1501498
Not Verified     759170
Name: count, dtype: int64

In [17]:
df_subset.drop(columns=['verification_status', 'merged_verification_status'], inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1903518293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns=['verification_status', 'merged_verification_status'], inplace=True)


In [18]:
df_subset.drop(columns=['dti', 'annual_inc', 'emp_length', 'purpose'])

,acc_now_delinq,all_util,avg_cur_bal,chargeoff_within_12_mths,delinq_2yrs,delinq_amnt,funded_amnt,home_ownership,int_rate,mths_since_last_delinq,...,total_bal_ex_mort,total_bc_limit,sec_app_chargeoff_within_12_mths,loan_status,grade,merged_annual_inc,merged_dti,merged_purpose,emp_length_grouped,clean_verification_status
member_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,28.0,1878.0,0.0,0.0,0.0,2500,RENT,13.56,NaN,...,16901.0,36500.0,NaN,Current,C,55000.0,18.24,Debt-related,10+ years,Not Verified
2,0.0,57.0,24763.0,0.0,0.0,0.0,30000,MORTGAGE,18.94,71.0,...,99468.0,15000.0,NaN,Current,D,90000.0,26.52,Debt-related,10+ years,Verified
3,0.0,35.0,18383.0,0.0,0.0,0.0,5000,MORTGAGE,17.97,NaN,...,11749.0,13800.0,NaN,Current,D,59280.0,10.51,Debt-related,4-6 years,Verified
4,0.0,70.0,30505.0,0.0,0.0,0.0,4000,MORTGAGE,18.94,NaN,...,36151.0,5000.0,NaN,Current,D,92000.0,16.74,Debt-related,10+ years,Verified
5,0.0,54.0,9667.0,0.0,0.0,0.0,30000,MORTGAGE,16.14,NaN,...,29674.0,9300.0,NaN,Current,C,57250.0,26.35,Debt-related,10+ years,Not Verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260664,0.0,38.0,4110.0,0.0,0.0,0.0,12000,MORTGAGE,14.08,NaN,...,45212.0,6200.0,NaN,Current,C,58000.0,20.88,Debt-related,10+ years,Not Verified
2260665,0.0,44.0,34640.0,0.0,3.0,0.0,12000,MORTGAGE,25.82,13.0,...,12928.0,11400.0,0.0,Fully Paid,E,135000.0,7.36,Debt-related,0-3 years,Not Verified
2260666,0.0,93.0,31623.0,0.0,0.0,0.0,10000,OWN,11.99,NaN,...,36448.0,2000.0,NaN,Current,B,64000.0,12.96,Debt-related,10+ years,Verified


In [19]:
df_subset.mths_since_last_delinq.unique()

array([ nan,  71.,  32.,  17.,  22.,   6.,  43.,  38.,   8.,  30.,  16.,
        24.,  37.,  52.,  62.,  36.,  10.,   7.,  57.,  56.,   4.,  21.,
        41.,  42.,  25.,   9.,  59.,  45.,  75.,  12.,  61.,  35.,  66.,
        51.,  15.,  73.,  29.,  80.,  39.,  11.,  27.,  13.,  55.,  74.,
        18.,  49.,  60.,  44.,  48.,  68.,  47.,  28.,   2.,  70.,  72.,
        65.,  19.,   1.,  46.,  34.,  81.,  77.,  69.,  40.,  33.,  20.,
         5.,  58.,  53.,  78., 100.,  64.,  23.,  63.,  67.,  14.,   3.,
       108.,  31.,  26.,  76.,  54.,  50., 134.,  79., 135.,  82., 113.,
        96.,  84., 107., 104., 115., 101.,  93., 153., 110.,  92.,  83.,
       106.,  86., 111., 130.,  97.,  89.,  95., 124., 119., 117., 120.,
        98.,  91.,  88., 105., 131.,  87., 109.,  94., 114., 116.,  99.,
       102., 150., 132., 146.,  90.,  85., 122., 128., 125., 133., 136.,
       118., 141., 127.,   0., 103., 160., 121., 151., 123., 162., 126.,
       112., 145., 161., 179., 138., 159., 147., 17

In [20]:
# Define a function to bucket values
def bucket_mths_since_last_delinq(value):
    if pd.isnull(value):  # Handle missing values
        return "Unknown"
    elif value <= 6:
        return "0-6 months"
    elif 7 <= value <= 12:
        return "7-12 months"
    elif 13 <= value <= 24:
        return "13-24 months"
    elif 25 <= value <= 60:
        return "25-60 months"
    else:
        return "61+ months"

# Apply the function to the column
df_subset['delinq_bucket'] = df_subset['mths_since_last_delinq'].apply(bucket_mths_since_last_delinq)

# View the updated DataFrame
print(df_subset[['mths_since_last_delinq', 'delinq_bucket']].head())

df_subset.delinq_bucket.value_counts()

           mths_since_last_delinq delinq_bucket
member_id                                      
1                             NaN       Unknown
2                            71.0    61+ months
3                             NaN       Unknown
4                             NaN       Unknown
5                             NaN       Unknown


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\2840876961.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['delinq_bucket'] = df_subset['mths_since_last_delinq'].apply(bucket_mths_since_last_delinq)


delinq_bucket
Unknown         1158502
25-60 months     492505
13-24 months     226410
61+ months       177712
7-12 months      119398
0-6 months        86141
Name: count, dtype: int64

In [21]:
df_subset.drop(columns='mths_since_last_delinq', inplace=True)
df_subset.head()

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\590793574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='mths_since_last_delinq', inplace=True)


,acc_now_delinq,all_util,annual_inc,avg_cur_bal,chargeoff_within_12_mths,delinq_2yrs,delinq_amnt,dti,emp_length,funded_amnt,...,total_bc_limit,sec_app_chargeoff_within_12_mths,loan_status,grade,merged_annual_inc,merged_dti,merged_purpose,emp_length_grouped,clean_verification_status,delinq_bucket
member_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,28.0,55000.0,1878.0,0.0,0.0,0.0,18.24,10+ years,2500,...,36500.0,NaN,Current,C,55000.0,18.24,Debt-related,10+ years,Not Verified,Unknown
2,0.0,57.0,90000.0,24763.0,0.0,0.0,0.0,26.52,10+ years,30000,...,15000.0,NaN,Current,D,90000.0,26.52,Debt-related,10+ years,Verified,61+ months
3,0.0,35.0,59280.0,18383.0,0.0,0.0,0.0,10.51,6 years,5000,...,13800.0,NaN,Current,D,59280.0,10.51,Debt-related,4-6 years,Verified,Unknown
4,0.0,70.0,92000.0,30505.0,0.0,0.0,0.0,16.74,10+ years,4000,...,5000.0,NaN,Current,D,92000.0,16.74,Debt-related,10+ years,Verified,Unknown
5,0.0,54.0,57250.0,9667.0,0.0,0.0,0.0,26.35,10+ years,30000,...,9300.0,NaN,Current,C,57250.0,26.35,Debt-related,10+ years,Not Verified,Unknown


In [22]:
print(df_subset.acc_now_delinq.value_counts())

def bucket_acc_now_delinq(value):
    if value == 0:
        return "No Delinquencies"
    else:  # For values 2 and higher, including 14.0
        return "At Least One Delinquency"

# Apply the bucketing function
df_subset['acc_now_delinq_bucketed'] = df_subset['acc_now_delinq'].apply(bucket_acc_now_delinq)

# View the updated DataFrame
print(df_subset[['acc_now_delinq', 'acc_now_delinq_bucketed']].head())

df_subset['acc_now_delinq_bucketed'].value_counts()



acc_now_delinq
0.0     2251857
1.0        8293
2.0         421
3.0          50
4.0          11
5.0           3
6.0           2
14.0          1
7.0           1
Name: count, dtype: int64
           acc_now_delinq acc_now_delinq_bucketed
member_id                                        
1                     0.0        No Delinquencies
2                     0.0        No Delinquencies
3                     0.0        No Delinquencies
4                     0.0        No Delinquencies
5                     0.0        No Delinquencies


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\413986859.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['acc_now_delinq_bucketed'] = df_subset['acc_now_delinq'].apply(bucket_acc_now_delinq)


acc_now_delinq_bucketed
No Delinquencies            2251857
At Least One Delinquency       8811
Name: count, dtype: int64

In [23]:
df_subset.drop(columns='acc_now_delinq', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\3999777910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='acc_now_delinq', inplace=True)


In [24]:
# Print the value counts for chargeoff_within_12_mths
print(df_subset['chargeoff_within_12_mths'].value_counts())

# Define the bucketing function for chargeoff_within_12_mths
def bucket_chargeoff_within_12_mths(value):
    if value == 0:
        return "No Chargeoffs Within 12 Months"
    else:  # For values 1 and higher
        return "At Least One Chargeoff Within 12 Months"

# Apply the bucketing function to the correct column
df_subset['chargeoff_within_12_mths_bucketed'] = df_subset['chargeoff_within_12_mths'].apply(bucket_chargeoff_within_12_mths)

# View the updated DataFrame
print(df_subset[['chargeoff_within_12_mths', 'chargeoff_within_12_mths_bucketed']].head())

# Get value counts for the bucketed column
print(df_subset['chargeoff_within_12_mths_bucketed'].value_counts())

chargeoff_within_12_mths
0.0     2243339
1.0       15765
2.0        1115
3.0         186
4.0          68
5.0          22
6.0          12
7.0           8
9.0           5
8.0           2
10.0          1
Name: count, dtype: int64
           chargeoff_within_12_mths chargeoff_within_12_mths_bucketed
member_id                                                            
1                               0.0    No Chargeoffs Within 12 Months
2                               0.0    No Chargeoffs Within 12 Months
3                               0.0    No Chargeoffs Within 12 Months
4                               0.0    No Chargeoffs Within 12 Months
5                               0.0    No Chargeoffs Within 12 Months
chargeoff_within_12_mths_bucketed
No Chargeoffs Within 12 Months             2243339
At Least One Chargeoff Within 12 Months      17329
Name: count, dtype: int64


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\2368612236.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['chargeoff_within_12_mths_bucketed'] = df_subset['chargeoff_within_12_mths'].apply(bucket_chargeoff_within_12_mths)


In [25]:
df_subset.drop(columns='chargeoff_within_12_mths', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1643959014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='chargeoff_within_12_mths', inplace=True)


In [26]:
def bucket_delinq_2yrs(value):
    if value == 0:
        return 0  # No delinquencies
    elif value == 1:
        return 1  # One delinquency
    elif value == 2:
        return 2  # Two delinquencies
    elif value == 3:
        return 3  # Three delinquencies
    else:  # For values 4 and higher
        return 4  # Four or more delinquencies

# Apply the bucketing function to the delinq_2yrs column
df_subset['delinq_2yrs_bucketed'] = df_subset['delinq_2yrs'].apply(bucket_delinq_2yrs)

# View the updated DataFrame
print(df_subset[['delinq_2yrs', 'delinq_2yrs_bucketed']].head())

# Get value counts for the bucketed column
print(df_subset['delinq_2yrs_bucketed'].value_counts())


           delinq_2yrs  delinq_2yrs_bucketed
member_id                                   
1                  0.0                     0
2                  0.0                     0
3                  0.0                     0
4                  0.0                     0
5                  0.0                     0
delinq_2yrs_bucketed
0    1839108
1     281353
2      81289
3      29542
4      29376
Name: count, dtype: int64


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\2314256977.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['delinq_2yrs_bucketed'] = df_subset['delinq_2yrs'].apply(bucket_delinq_2yrs)


In [27]:
df_subset.drop(columns='delinq_2yrs', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\2893621755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='delinq_2yrs', inplace=True)


In [28]:
df_subset.head()

,all_util,annual_inc,avg_cur_bal,delinq_amnt,dti,emp_length,funded_amnt,home_ownership,int_rate,num_tl_90g_dpd_24m,...,grade,merged_annual_inc,merged_dti,merged_purpose,emp_length_grouped,clean_verification_status,delinq_bucket,acc_now_delinq_bucketed,chargeoff_within_12_mths_bucketed,delinq_2yrs_bucketed
member_id,,,,,,,,,,,,,,,,,,,,,
1,28.0,55000.0,1878.0,0.0,18.24,10+ years,2500,RENT,13.56,0.0,...,C,55000.0,18.24,Debt-related,10+ years,Not Verified,Unknown,No Delinquencies,No Chargeoffs Within 12 Months,0
2,57.0,90000.0,24763.0,0.0,26.52,10+ years,30000,MORTGAGE,18.94,0.0,...,D,90000.0,26.52,Debt-related,10+ years,Verified,61+ months,No Delinquencies,No Chargeoffs Within 12 Months,0
3,35.0,59280.0,18383.0,0.0,10.51,6 years,5000,MORTGAGE,17.97,0.0,...,D,59280.0,10.51,Debt-related,4-6 years,Verified,Unknown,No Delinquencies,No Chargeoffs Within 12 Months,0
4,70.0,92000.0,30505.0,0.0,16.74,10+ years,4000,MORTGAGE,18.94,0.0,...,D,92000.0,16.74,Debt-related,10+ years,Verified,Unknown,No Delinquencies,No Chargeoffs Within 12 Months,0
5,54.0,57250.0,9667.0,0.0,26.35,10+ years,30000,MORTGAGE,16.14,0.0,...,C,57250.0,26.35,Debt-related,10+ years,Not Verified,Unknown,No Delinquencies,No Chargeoffs Within 12 Months,0


In [29]:
print(df_subset.num_tl_90g_dpd_24m.value_counts())

def bucket_num_tl_90g_dpd_24m(value):
    if value == 0:
        return 0  # No occurrences
    else:
        return 1  # At least one occurrence

# Apply the bucketing function to the num_tl_90g_dpd_24m column
df_subset['accounts_90d_past_due_24m'] = df_subset['num_tl_90g_dpd_24m'].apply(bucket_num_tl_90g_dpd_24m)

# Get value counts for the bucketed column
print(df_subset['accounts_90d_past_due_24m'].value_counts())

num_tl_90g_dpd_24m
0.0     2073060
1.0       88450
2.0       16897
3.0        4672
4.0        2653
5.0        1398
6.0        1018
7.0         604
8.0         467
9.0         340
10.0        225
11.0        170
12.0        125
13.0         78
14.0         69
15.0         49
16.0         29
18.0         19
20.0         15
17.0         14
19.0         11
21.0          6
22.0          6
30.0          3
26.0          3
23.0          2
24.0          2
42.0          1
25.0          1
35.0          1
39.0          1
29.0          1
58.0          1
36.0          1
Name: count, dtype: int64
accounts_90d_past_due_24m
0    2073060
1     187608
Name: count, dtype: int64


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\3426180600.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['accounts_90d_past_due_24m'] = df_subset['num_tl_90g_dpd_24m'].apply(bucket_num_tl_90g_dpd_24m)


In [30]:
df_subset.drop(columns = 'num_tl_90g_dpd_24m', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1640642007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns = 'num_tl_90g_dpd_24m', inplace=True)


In [31]:
df_subset.sec_app_chargeoff_within_12_mths.value_counts()

sec_app_chargeoff_within_12_mths
0.0     105117
1.0       2073
2.0        429
3.0        146
4.0         90
5.0         57
6.0         30
7.0         20
10.0        14
8.0         12
9.0         10
11.0         5
12.0         3
13.0         2
18.0         2
20.0         2
14.0         2
21.0         2
17.0         2
19.0         1
15.0         1
16.0         1
Name: count, dtype: int64

In [32]:
# Transform sec_app_chargeoff_within_12_mths to 0 or 1
df_subset['sec_app_chargeoff_within_12_mths'] = df_subset['sec_app_chargeoff_within_12_mths'].apply(lambda x: 1 if x > 0 else 0)

# Print value counts to confirm the transformation
print(df_subset['sec_app_chargeoff_within_12_mths'].value_counts())

# Convert 'chargeoff_within_12_mths_bucketed' to 0 and 1
df_subset['chargeoff_within_12_mths'] = df_subset['chargeoff_within_12_mths_bucketed'].map({
    "No Chargeoffs Within 12 Months": 0,
    "At Least One Chargeoff Within 12 Months": 1
})

# Merge both columns by summing their values row-wise
df_subset['total_chargeoffs_within_12_mths'] = df_subset['chargeoff_within_12_mths'] + df_subset['sec_app_chargeoff_within_12_mths']

sec_app_chargeoff_within_12_mths
0    2257764
1       2904
Name: count, dtype: int64


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1752401267.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['sec_app_chargeoff_within_12_mths'] = df_subset['sec_app_chargeoff_within_12_mths'].apply(lambda x: 1 if x > 0 else 0)
C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1752401267.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['chargeoff_within_12_mths'] = df_subset['chargeoff_within_12_mths_bucketed'].map({
C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1752401267.py:

In [33]:
df_subset.head()

,all_util,annual_inc,avg_cur_bal,delinq_amnt,dti,emp_length,funded_amnt,home_ownership,int_rate,open_acc,...,merged_purpose,emp_length_grouped,clean_verification_status,delinq_bucket,acc_now_delinq_bucketed,chargeoff_within_12_mths_bucketed,delinq_2yrs_bucketed,accounts_90d_past_due_24m,chargeoff_within_12_mths,total_chargeoffs_within_12_mths
member_id,,,,,,,,,,,,,,,,,,,,,
1,28.0,55000.0,1878.0,0.0,18.24,10+ years,2500,RENT,13.56,9.0,...,Debt-related,10+ years,Not Verified,Unknown,No Delinquencies,No Chargeoffs Within 12 Months,0,0,0,0
2,57.0,90000.0,24763.0,0.0,26.52,10+ years,30000,MORTGAGE,18.94,13.0,...,Debt-related,10+ years,Verified,61+ months,No Delinquencies,No Chargeoffs Within 12 Months,0,0,0,0
3,35.0,59280.0,18383.0,0.0,10.51,6 years,5000,MORTGAGE,17.97,8.0,...,Debt-related,4-6 years,Verified,Unknown,No Delinquencies,No Chargeoffs Within 12 Months,0,0,0,0
4,70.0,92000.0,30505.0,0.0,16.74,10+ years,4000,MORTGAGE,18.94,10.0,...,Debt-related,10+ years,Verified,Unknown,No Delinquencies,No Chargeoffs Within 12 Months,0,0,0,0
5,54.0,57250.0,9667.0,0.0,26.35,10+ years,30000,MORTGAGE,16.14,12.0,...,Debt-related,10+ years,Not Verified,Unknown,No Delinquencies,No Chargeoffs Within 12 Months,0,0,0,0


In [34]:
df_subset.drop(columns = ['emp_length','chargeoff_within_12_mths', 'sec_app_chargeoff_within_12_mths', 'chargeoff_within_12_mths_bucketed'], inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\3162556841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns = ['emp_length','chargeoff_within_12_mths', 'sec_app_chargeoff_within_12_mths', 'chargeoff_within_12_mths_bucketed'], inplace=True)


In [35]:
df_subset.drop(columns = ['dti'], inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\287521743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns = ['dti'], inplace=True)


In [36]:
df_subset['open_acc'].describe()

count    2.260639e+06
mean     1.161240e+01
std      5.640861e+00
min      0.000000e+00
25%      8.000000e+00
50%      1.100000e+01
75%      1.400000e+01
max      1.010000e+02
Name: open_acc, dtype: float64

In [37]:
def bucket_open_acc(value):
    if pd.isnull(value):  # Handle missing values
        return "Unknown"
    elif value == 0:
        return "0"
    elif 1 <= value <= 5:
        return "1-5"
    elif 6 <= value <= 10:
        return "6-10"
    elif 11 <= value <= 15:
        return "11-15"
    elif 16 <= value <= 20:
        return "16-20"
    else:  # For values greater than 20
        return "21+"

# Apply the bucketing function to the open_acc column
df_subset['open_acc_bucketed'] = df_subset['open_acc'].apply(bucket_open_acc)

# View the updated DataFrame
print(df_subset[['open_acc', 'open_acc_bucketed']].head())

# Get value counts for the bucketed column
print(df_subset['open_acc_bucketed'].value_counts())

           open_acc open_acc_bucketed
member_id                            
1               9.0              6-10
2              13.0             11-15
3               8.0              6-10
4              10.0              6-10
5              12.0             11-15
open_acc_bucketed
6-10       892494
11-15      688007
16-20      296765
1-5        221324
21+        161993
0              56
Unknown        29
Name: count, dtype: int64


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\3065553958.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['open_acc_bucketed'] = df_subset['open_acc'].apply(bucket_open_acc)


In [38]:
df_subset.drop(columns='open_acc', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\678917131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='open_acc', inplace=True)


In [39]:
df_subset.pct_tl_nvr_dlq.value_counts()
df_subset.pct_tl_nvr_dlq.describe()

count    2.190237e+06
mean     9.411458e+01
std      9.036140e+00
min      0.000000e+00
25%      9.130000e+01
50%      1.000000e+02
75%      1.000000e+02
max      1.000000e+02
Name: pct_tl_nvr_dlq, dtype: float64

In [40]:
def bucket_pct_tl_nvr_dlq(value):
    if pd.isnull(value):  # Handle missing values
        return "Unknown"
    elif value == 100:
        return "100.0"
    elif 90 <= value < 100:
        return "90-99.9"
    elif 80 <= value < 90:
        return "80-89.9"
    else:  # For values below 80
        return "<80"

# Apply the bucketing function to the pct_tl_nvr_dlq column
df_subset['pct_tl_nvr_dlq_bucketed'] = df_subset['pct_tl_nvr_dlq'].apply(bucket_pct_tl_nvr_dlq)

# View the updated DataFrame
print(df_subset[['pct_tl_nvr_dlq', 'pct_tl_nvr_dlq_bucketed']].head())

# Get value counts for the bucketed column
print(df_subset['pct_tl_nvr_dlq_bucketed'].value_counts())

           pct_tl_nvr_dlq pct_tl_nvr_dlq_bucketed
member_id                                        
1                   100.0                   100.0
2                    95.0                 90-99.9
3                   100.0                   100.0
4                   100.0                   100.0
5                    92.3                 90-99.9
pct_tl_nvr_dlq_bucketed
100.0      1106209
90-99.9     621641
80-89.9     294229
<80         168158
Unknown      70431
Name: count, dtype: int64


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\3632145594.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['pct_tl_nvr_dlq_bucketed'] = df_subset['pct_tl_nvr_dlq'].apply(bucket_pct_tl_nvr_dlq)


In [41]:
df_subset.drop(columns='pct_tl_nvr_dlq', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\482067499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='pct_tl_nvr_dlq', inplace=True)


In [42]:
print(df_subset.percent_bc_gt_75.value_counts())
print(df_subset.percent_bc_gt_75.describe())

percent_bc_gt_75
0.0      598711
100.0    372700
50.0     231935
33.3     147284
66.7     134722
          ...  
95.8          1
37.0          1
26.9          1
26.5          1
2.4           1
Name: count, Length: 284, dtype: int64
count    2.185289e+06
mean     4.243513e+01
std      3.621616e+01
min      0.000000e+00
25%      0.000000e+00
50%      3.750000e+01
75%      7.140000e+01
max      1.000000e+02
Name: percent_bc_gt_75, dtype: float64


In [43]:
def bucket_percent_bc_gt_75(value):
    if pd.isnull(value):  # Handle missing values
        return "Unknown"
    elif value == 0:
        return "0"
    elif 0 < value <= 25:
        return "1-25%"
    elif 25 < value <= 50:
        return "25.1-50%"
    elif 50 < value <= 75:
        return "50.1-75%"
    else:  # For values greater than 75
        return "75.1-100%"

# Apply the bucketing function to the percent_bc_gt_75 column
df_subset['percent_bc_gt_75_bucketed'] = df_subset['percent_bc_gt_75'].apply(bucket_percent_bc_gt_75)

# View the updated DataFrame
print(df_subset[['percent_bc_gt_75', 'percent_bc_gt_75_bucketed']].head())

# Get value counts for the bucketed column
print(df_subset['percent_bc_gt_75_bucketed'].value_counts())

           percent_bc_gt_75 percent_bc_gt_75_bucketed
member_id                                            
1                       0.0                         0
2                       0.0                         0
3                       0.0                         0
4                     100.0                 75.1-100%
5                       0.0                         0
percent_bc_gt_75_bucketed
0            598711
25.1-50%     510791
75.1-100%    463819
50.1-75%     308926
1-25%        303042
Unknown       75379
Name: count, dtype: int64


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1278980967.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['percent_bc_gt_75_bucketed'] = df_subset['percent_bc_gt_75'].apply(bucket_percent_bc_gt_75)


In [44]:
df_subset.drop(columns='percent_bc_gt_75', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\2313790535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='percent_bc_gt_75', inplace=True)


In [45]:
df_subset.drop(columns='purpose', inplace=True)


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1514372739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='purpose', inplace=True)


In [46]:
print(df_subset.pub_rec_bankruptcies.value_counts())
print(df_subset.pub_rec_bankruptcies.describe())

pub_rec_bankruptcies
0.0     1987383
1.0      258444
2.0       10518
3.0        2131
4.0         541
5.0         188
6.0          60
7.0          23
8.0          10
9.0           3
11.0          1
12.0          1
Name: count, dtype: int64
count    2.259303e+06
mean     1.281935e-01
std      3.646130e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.200000e+01
Name: pub_rec_bankruptcies, dtype: float64


In [47]:
def bucket_pub_rec_bankruptcies(value):
    if value == 0:
        return 0  # No bankruptcies
    else:
        return 1  # At least one bankruptcy

# Apply the bucketing function to the pub_rec_bankruptcies column
df_subset['pub_rec_bankruptcies_bucketed'] = df_subset['pub_rec_bankruptcies'].apply(bucket_pub_rec_bankruptcies)

# View the updated DataFrame
print(df_subset[['pub_rec_bankruptcies', 'pub_rec_bankruptcies_bucketed']].head())

# Get value counts for the bucketed column
print(df_subset['pub_rec_bankruptcies_bucketed'].value_counts())

           pub_rec_bankruptcies  pub_rec_bankruptcies_bucketed
member_id                                                     
1                           1.0                              1
2                           1.0                              1
3                           0.0                              0
4                           0.0                              0
5                           0.0                              0
pub_rec_bankruptcies_bucketed
0    1987383
1     273285
Name: count, dtype: int64


C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\829383623.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['pub_rec_bankruptcies_bucketed'] = df_subset['pub_rec_bankruptcies'].apply(bucket_pub_rec_bankruptcies)


In [48]:
df_subset.drop(columns='pub_rec_bankruptcies', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\4050585364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='pub_rec_bankruptcies', inplace=True)


In [49]:
df_subset.drop(columns='annual_inc', inplace=True)

C:\Users\seyhr\AppData\Local\Temp\ipykernel_16012\1336946345.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.drop(columns='annual_inc', inplace=True)


In [50]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2260668 entries, 1 to 2260668
Data columns (total 27 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   all_util                         float64
 1   avg_cur_bal                      float64
 2   delinq_amnt                      float64
 3   funded_amnt                      int64  
 4   home_ownership                   object 
 5   int_rate                         float64
 6   term                             object 
 7   tot_hi_cred_lim                  float64
 8   total_acc                        float64
 9   total_bal_ex_mort                float64
 10  total_bc_limit                   float64
 11  loan_status                      object 
 12  grade                            object 
 13  merged_annual_inc                float64
 14  merged_dti                       float64
 15  merged_purpose                   object 
 16  emp_length_grouped               object 
 17  clean_verific

In [51]:
# Export df_subset to a CSV file
df_subset.to_csv('../../resources/df_subset.csv', index=True)

print("DataFrame successfully exported to 'df_subset.csv'")

DataFrame successfully exported to 'df_subset.csv'
